# Problem Set 7

Franklin She, Selina Ni

## Problem 1



### 1.

If we only observe $(Y, \tilde{X}, \u{X})$, what we estimate is

$$ Y = \tilde{X}\beta - V^1 \beta + U $$

Taking $\tilde{U} = - V^1 \beta + U$, we have

$$ Y = \tilde{X}\beta + \tilde{U} $$

In this model, $\u{X}$ is exogenous (covariance between $\u{X}$ and the error term is zero) because of assumptions (a) - (d) and assuming $E[XU] = 0$.

$$
\begin{aligned}
Cov(\u{X}, \tilde{U}) &= Cov\left(X + V^2, - V^1 \beta + U\right) \\
&= -\beta Cov(X, V^1) - \beta Cov(V^2, V^1) + Cov(X, U) + Cov(V^2, U) \\
&= 0
\end{aligned}
$$

In this model $\u{X}$ is relevant (covariance between $\u{X}$ and $\tilde{X}$ is nonzero) because of assumption (b) and (d) and assuming $Var(X) > 0$.

$$
\begin{aligned}
Cov(\u{X}, \tilde{X}) &= Cov\left(X + V^2, X + V^1\right) \\
&= Cov(X, X) + Cov(X, V^1) + Cov(V^2, X) + Cov(V^2, V^1) \\
&= Var(x) \\
& \neq 0
\end{aligned}
$$

Therefore $\u{X}$ is an instrument for $\tilde{X}$ in this model.

### 2.

#### (a)

We can expect $\beta^s > 0$ by the law of supply, and $\beta^d < 0$ by the law of demand. It follows that we can expect $\beta^d - \beta^s < 0$.

#### (b)

Setting $Q^d(P) = Q^s(P)$:

$$
\begin{aligned}
    \alpha^d + P \beta^d + U^d &= \alpha^s + P \beta^s + Z \gamma + U^s\\
    P \beta^d - P \beta^s &= \alpha^s - \alpha^d + Z \gamma + U^s - U^d \\
    P &= \frac{\alpha^s - \alpha^d + Z \gamma + U^s - U^d}{\beta^d - \beta^s}
\end{aligned}
$$

#### (c)

The model for demand, $Q^d(P) = \alpha^d + P \beta^d + U^d$, in equilibrium, becomes $Q^{eq} = \alpha^d + P^{eq} \beta^d + U^d$. Consider $Cov(P^{eq}, U^d)$.

$$
\begin{aligned}
    Cov(P^{eq}, U^d) &= Cov\left(\frac{\alpha^s - \alpha^d + Z \gamma + U^s - U^d}{\beta^d - \beta^s}, U^d\right) \\
    &= - \frac{1}{\beta^d - \beta^s} Var(U^d) \\
    &\neq 0
\end{aligned}
$$

Therefore, $P^{eq}$ is endogenous.

#### (d)

In the model for demand, $Z$ is exogenous (covariance between $Z$ and the error term is zero) because we assume $E[ZU^d] = 0$.

In the model for demand, $Z$ is relevant (covariance between $Z$ and $P^{eq}$ is not zero), because we assume $\gamma \neq 0$ and $Var(Z) > 0$

$$
\begin{aligned}
    Cov(P^{eq}, Z) &= Cov\left(\frac{\alpha^s - \alpha^d + Z \gamma + U^s - U^d}{\beta^d - \beta^s}, Z\right) \\
    &= \frac{\gamma}{\beta^d - \beta^s} Var(Z) \\
    &\neq 0
\end{aligned}
$$

Therefore $Z$ is an instrument for $P$ in the demand model.

#### (e)

In the model for supply, $Z$ is exogenous (covariance between $Z$ and the error term is zero) because we assume $E[ZU^s] = 0$.

$Z$ remains relevant by the derivation in part (d), so $Z$ is also an instrument for $P$ in the supply model.

## Problem 2

In [1]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [3]:
# 1.(a-c)

def generate_simulation_data(n, beta1, beta2, beta3=0, beta4=0, gamma1=0, gamma2=0):
    """
    Generate simulation data based on specified parameters.
    """
    x2 = np.random.normal(0, 1, n)
    v = np.random.normal(0, 0.1, n)
    x1 = x2 + x2**2 * gamma1 + x2**5 * gamma2 + v
    u = np.random.normal(0, 1, n)
    y = x1*beta1 + x2*beta2 + x2**2*beta3 + np.sin(x2)*beta4 + u
    
    return y, x1, x2


In [27]:
# 2.(a-c)

def estimate_coefficients(y, x1, x2, verbose = False):
    """
    Estimates coefficients and standard errors for unrestricted, restricted, and pretest regressions.
    """
    results = {}

    # Unrestricted Regression
    X_unrestricted = sm.add_constant(np.column_stack((x1, x2)))
    model_unrestricted = sm.OLS(y, X_unrestricted).fit()
    results['unrestricted'] = {
        'beta1': model_unrestricted.params[1],
        'std_err': model_unrestricted.bse[1],
        'ci': model_unrestricted.conf_int(alpha=0.05, cols=None)[1]
    }

    # Restricted Regression
    X_restricted = sm.add_constant(x1)
    model_restricted = sm.OLS(y, X_restricted).fit()
    results['restricted'] = {
        'beta1': model_restricted.params[1],
        'std_err': model_restricted.bse[1],
        'ci': model_restricted.conf_int(alpha=0.05, cols=None)[1]
    }

    # Pretest Regression
    p_value = model_unrestricted.pvalues[2]  # Test for beta2 = 0 in unrestricted model
    if p_value > 0.05:  # Using a significance level of 0.05
        results['pretest'] = {
            'beta1': results['restricted']['beta1'],
            'std_err': results['restricted']['std_err'],
            'ci': results['restricted']['ci'],
            'model_selected': 'restricted'
        }
    else:
        results['pretest'] = {
            'beta1': results['unrestricted']['beta1'],
            'std_err': results['unrestricted']['std_err'],
            'ci': results['unrestricted']['ci'],
            'model_selected': 'unrestricted'
        }

    if verbose:
        print(model_unrestricted.summary())
        print(model_restricted.summary())
        print(f'p-value for pretest: {p_value}')

    return results


In [15]:
# 3.(a)

S = 1000  # Number of simulations per n value
n_values = [10, 100, 1000, 10000]
beta1 = 2

# Placeholder for simulation results
simulation_data = {n: [] for n in n_values}

for n in n_values:
    beta2 = 3 * np.sqrt(n)
    for s in range(S):
        y, x1, x2 = generate_simulation_data(n, beta1, beta2)
        simulation_data[n].append((y, x1, x2))


In [32]:
# 3.(b)

# Placeholder for estimator results
estimator_results = {n: {'unrestricted': [], 'restricted': [], 'pretest': []} for n in n_values}

for n in n_values:
    for s in range(S):
        # Extract y, x1, and x2 from simulation results
        y, x1, x2 = simulation_data[n][s]
        
        # Compute the estimators for the current dataset
        estimates = estimate_coefficients(y, x1, x2)
        
        # Store the results
        estimator_results[n]['unrestricted'].append(estimates['unrestricted'])
        estimator_results[n]['restricted'].append(estimates['restricted'])
        estimator_results[n]['pretest'].append(estimates['pretest'])


In [39]:
# 4.(a)

# Initialize lists to store summary statistics
summary_stats = []

# Iterate over each n to calculate mean and variance for each estimator
for n in n_values:
    for method in ['unrestricted', 'restricted', 'pretest']:
        estimates = [est['beta1'] for est in estimator_results[n][method]]
        std_errors = [est['std_err'] for est in estimator_results[n][method]]

        # Calculate mean and variance of beta1 estimates
        mean_beta1 = np.mean(estimates)
        variance_beta1 = np.var(estimates)
        
        # Append summary statistics
        summary_stats.append({
            'n': n,
            'Method': method,
            'Mean of Beta1': mean_beta1,
            'Variance of Beta1': variance_beta1
        })

# Convert summary statistics to a DataFrame for easy visualization
summary_df = pd.DataFrame(summary_stats)

print(summary_df)


        n        Method  Mean of Beta1  Variance of Beta1
0      10  unrestricted       1.931861          18.162193
1      10    restricted      11.366193           0.257339
2      10       pretest       4.774116          39.039461
3     100  unrestricted       2.009798           1.030558
4     100    restricted      31.685454           0.103970
5     100       pretest       2.009798           1.030558
6    1000  unrestricted       2.000075           0.100508
7    1000    restricted      95.918828           0.092784
8    1000       pretest       2.000075           0.100508
9   10000  unrestricted       1.999880           0.008628
10  10000    restricted     299.028230           0.084317
11  10000       pretest       1.999880           0.008628


We see that the mean of $\beta_1$ for the unrestricted model is very accurate (because we include everything in the model), adn the variance of this estimate decreases as $n$ increases.

We also see that the mean of $\beta_1$ for the restricted model increases dramatically, because as $n$ increases, our true $\beta_2$ increases (by our definition), and the $\beta_1$ estimate captures this.

In [42]:
# 4.(b)

# Initialize a list to store model selection frequencies
model_selection_stats = []

# Iterate over each n to calculate the frequency of correct model selection
for n in n_values:
    selected_correct_model = [1 for est in estimator_results[n]['pretest'] if est['model_selected'] == 'unrestricted']
    fraction_correct = sum(selected_correct_model) / S
    model_selection_stats.append({'n': n, 'Fraction Correct': fraction_correct})

# Convert to DataFrame and print
model_selection_df = pd.DataFrame(model_selection_stats)
print(model_selection_df)


       n  Fraction Correct
0     10             0.575
1    100             1.000
2   1000             1.000
3  10000             1.000


As $n$ gets larger, the fraction that the pretest selects the unrestricted true model gets larger as well. This is also because how we define $\beta_2$, it gets larger as $n$ gets larger.
The p-value for the significance for $\beta_2$ probably increases as $n$ gets larger, meaning that the pretest will select the unrestricted true model more often. 

In [43]:
# True value of beta1
true_beta1 = 2

# Initialize list to store CI coverage and length stats
ci_stats = []

for n in n_values:
    for method in ['unrestricted', 'restricted', 'pretest']:
        estimates = estimator_results[n][method]
        # Calculate CI coverage and average length (using 0.05 CI directly from statsmodels)
        ci_coverages = [(true_beta1 >= est['ci'][0]) and (true_beta1 <= est['ci'][1]) for est in estimates]
        ci_lengths = [est['ci'][1] - est['ci'][0] for est in estimates]
        
        ci_stats.append({
            'n': n,
            'Method': method,
            'CI Coverage': sum(ci_coverages) / S,
            'Average CI Length': np.mean(ci_lengths)
        })

ci_stats_df = pd.DataFrame(ci_stats)
print(ci_stats_df)

        n        Method  CI Coverage  Average CI Length
0      10  unrestricted        0.947          18.013109
1      10    restricted        0.000           2.228009
2      10       pretest        0.545           9.414855
3     100  unrestricted        0.951           4.044289
4     100    restricted        0.000           1.249668
5     100       pretest        0.951           4.044289
6    1000  unrestricted        0.947           1.243827
7    1000    restricted        0.000           1.172165
8    1000       pretest        0.947           1.243827
9   10000  unrestricted        0.966           0.392130
10  10000    restricted        0.000           1.165308
11  10000       pretest        0.966           0.392130


The results are that the unrestricted model CI includes the true value of beta around 95% of the time, while the restricted model CI never includes the true value of beta. As we saw earlier, the pretest always seems to select the unrestricted model when $n \in \{100, 1000, 10000\}$, and selects the restricted model about half the time when $n = 10$.

This is consistent with what we would expect. Because $x_1$ is extremely correlated with $x_2$, not including it in the model leads to very bad estimates of $\beta_1$ (restricted model).